In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input


# ============================================================================
# 0. Configuración para Mac con TensorFlow-MacOS y TensorFlow-Metal
# ============================================================================
# Configuración de GPU 
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"Se detectó {len(gpus)} GPU(s). ¡Excelente, Papi!")
        for gpu in gpus:
            # Activa el crecimiento de la memoria en GPU
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        print("No se detectó GPU. Verifica que estés usando 'tensorflow-macos' y 'tensorflow-metal', Daddy.")
except Exception as e:
    print("Error en la configuración de la GPU:", e)
# ============================================================================
# 1. Lectura y preparación de datos
# ============================================================================
# Rutas locales a tus CSV y carpetas de imágenes
TRAIN_IMG_DIR = "./tiny-imagenet-200/train/"
VAL_IMG_DIR = "./tiny-imagenet-200/val/images/"
TEST_IMG_DIR = "./tiny-imagenet-200/test/images/"

# Rutas a los CSV
TRAIN_CSV_PATH = "./train_data.csv"
VAL_CSV_PATH = "./val_data.csv"
TEST_CSV_PATH = "./test_data.csv"

# Lee los CSV
train_df = pd.read_csv(TRAIN_CSV_PATH)
val_df = pd.read_csv(VAL_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)
#Ajuste en el path de train_df
train_df['File'] = train_df['File'].str.replace('/kaggle/input/tiny-imagenet/tiny-imagenet-200/train/', '')
train_df.head()

Se detectó 1 GPU(s). ¡Excelente, Papi!


,File,Encoded_Label
0,n02437312/images/n02437312_229.JPEG,53
1,n02437312/images/n02437312_394.JPEG,53
2,n02437312/images/n02437312_15.JPEG,53
3,n02437312/images/n02437312_62.JPEG,53
4,n02437312/images/n02437312_17.JPEG,53


In [4]:
# ============================================================================
# 2. Data Augmentation
# ============================================================================
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [ ]:
train_df['Encoded_Label'] = train_df['Encoded_Label'].astype(str)
val_df['Encoded_Label'] = val_df['Encoded_Label'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='File',
    y_col='Encoded_Label',
    target_size=(64, 64),
    class_mode='categorical',
    batch_size=64,
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='File',
    y_col='Encoded_Label',
    target_size=(64, 64),
    class_mode='categorical',
    batch_size=64,
    shuffle=False
)

test_generator = val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='File',
    y_col=None,
    target_size=(64, 64),
    class_mode=None,
    batch_size=64,
    shuffle=False
)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames.


/Users/tomasduque21/Documents/Tomás/Maestria /Aprendizaje Automatico/Competicion 2/ven/lib/python3.9/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 100000 invalid image filename(s) in x_col="File". These filename(s) will be ignored.
  warnings.warn(
/Users/tomasduque21/Documents/Tomás/Maestria /Aprendizaje Automatico/Competicion 2/ven/lib/python3.9/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 6000 invalid image filename(s) in x_col="File". These filename(s) will be ignored.
  warnings.warn(
/Users/tomasduque21/Documents/Tomás/Maestria /Aprendizaje Automatico/Competicion 2/ven/lib/python3.9/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 4000 invalid image filename(s) in x_col="File". These filename(s) will be ignored.
  warnings.warn(
